In [6]:
# from sklearn.metrics import f1_score
# # 一、导入数据
# # -- coding: utf-8 --
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# from sklearn.model_selection import train_test_split
# import pandas as pd
# import numpy as np
# import jieba
# import torch
# import warnings
# from torch.optim import AdamW
# from torch.optim.lr_scheduler import ReduceLROnPlateau
# from sklearn.metrics import accuracy_score, precision_score, recall_score
# import os
#
# warnings.filterwarnings("ignore")  # 忽略警告信息
#
# # 加载语料库文件，并导入数据
# neg = pd.read_excel('data/neg.xls', header=None)
# pos = pd.read_excel('data/pos.xls', header=None)
#
# pos.head()
#
# # 分词处理
# word_cut = lambda x: jieba.lcut(str(x))
# pos['words'] = pos[0].apply(word_cut)
# neg['words'] = neg[0].apply(word_cut)
#
# # 使用 1 表示积极情绪，0 表示消极情绪，并完成数组拼接
# texts = np.concatenate((pos['words'], neg['words']))
# labels = np.concatenate((np.ones(len(pos)), np.zeros(len(neg))))
#
# # 二、使用 BERT 进行情感分类
# # 加载预训练的 BERT 模型和分词器
# tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
# model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=2)
#
# # 将分词后的句子转换为 BERT 的输入格式
# def encode_text(texts):
#     return tokenizer(texts,
#                      return_tensors='pt',
#                      padding=True,
#                      truncation=True,
#                      max_length=128)
#
# # 准备训练数据
# train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)
#
# train_encodings = encode_text([' '.join(text) for text in train_texts])
# val_encodings = encode_text([' '.join(text) for text in val_texts])
#
# # 将数据转换为 PyTorch 数据集
# class SentimentDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels
#
#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx], dtype=torch.long) for key, val in self.encodings.items()}  # 确保索引为整数类型
#         item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
#         return item
#
#     def __len__(self):
#         return len(self.labels)
#
# train_dataset = SentimentDataset(train_encodings, train_labels)
# val_dataset = SentimentDataset(val_encodings, val_labels)
#
#
# # 自定义计算 F1 值的函数
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=1)
#     f1 = f1_score(labels, predictions, average='binary')  # 二分类问题使用 'binary'
#     return {"f1": f1}
#
# # def compute_metrics(eval_pred):
# #     logits, labels = eval_pred
# #     predictions = np.argmax(logits, axis=1)
# #     accuracy = accuracy_score(labels, predictions)
# #     precision = precision_score(labels, predictions, average='binary')
# #     recall = recall_score(labels, predictions, average='binary')
# #     f1 = f1_score(labels, predictions, average='binary')
# #     return {
# #         "accuracy": accuracy,
# #         "precision": precision,
# #         "recall": recall,
# #         "f1": f1
# #     }
# # 设置优化器和学习率调整器
# optimizer = AdamW(model.parameters(), lr=2e-5)  # 初始学习率
# scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True, min_lr=1e-7)
#
# # 三、训练 BERT 情感分类模型
# # 设置训练参数
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=3,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=64,
#     warmup_steps=500,
#     weight_decay=0.01,
#     evaluation_strategy="epoch",  # 每个 epoch 结束时进行评估
#     logging_dir='./logs',
#     lr_scheduler_type="reduce_lr_on_plateau",  # 使用 ReduceLROnPlateau 调度器
#     metric_for_best_model="loss",  # 使用损失值作为最佳模型的评估指标
# )
# # training_args = TrainingArguments(
# #     output_dir='./results',
# #     num_train_epochs=3,
# #     per_device_train_batch_size=16,
# #     per_device_eval_batch_size=64,
# #     warmup_steps=500,
# #     weight_decay=0.01,
# #     evaluation_strategy="epoch",  # 每个 epoch 结束时进行评估
# #     logging_dir='./logs',
# #     lr_scheduler_type="reduce_lr_on_plateau",  # 使用 ReduceLROnPlateau 调度器
# #     metric_for_best_model="f1",  # 使用 F1 Score 作为最佳模型的评估指标
# # )
# # 初始化 Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     compute_metrics=compute_metrics,
# )
#
# # 开始训练
# trainer.train()
#
# # 输出训练后的指标
# eval_results = trainer.evaluate()
# print("Training Results:")
# # print(f"Accuracy: {eval_results['eval_accuracy']:.4f}")
# # print(f"Precision: {eval_results['eval_precision']:.4f}")
# # print(f"Recall: {eval_results['eval_recall']:.4f}")
# print(f"F1 Score: {eval_results['eval_f1']:.4f}")
#
# # 创建表格并打印
# df = pd.DataFrame([eval_results])
# print("\nEvaluation Metrics:")
# print(df.to_markdown(index=False))
# # 保存训练好的模型
# model.save_pretrained('bert_sentiment_model')
# tokenizer.save_pretrained('bert_sentiment_model')
#
# # 四、情感预测
# # 加载训练好的模型
# model = BertForSequenceClassification.from_pretrained('bert_sentiment_model')
# tokenizer = BertTokenizer.from_pretrained('bert_sentiment_model')
#
# # 检查设备
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# model = model.to(device)
#
# # 对电影评论进行情感判断
# def bert_predict(string):
#     # 对输入文本进行编码
#     inputs = tokenizer(string, return_tensors='pt', padding=True, truncation=True, max_length=128)
#     inputs = {k: v.to(device) for k, v in inputs.items()}  # 确保输入张量在正确的设备上
#     with torch.no_grad():
#         outputs = model(**inputs)
#         logits = outputs.logits
#         predicted_class = torch.argmax(logits).item()
#
#     # 输出结果
#     sentiment = '积极' if predicted_class == 1 else '消极'
#     print(f"{string} [{sentiment}]")
#     return sentiment
#
# # 测试预测
# string = '还不错，符合需求'
# pred_result = bert_predict(string)
# print(f"预测结果: {pred_result}")

In [7]:
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# from sklearn.model_selection import train_test_split
# import pandas as pd
# import numpy as np
# import jieba
# import torch
# from torch.optim import AdamW
# from torch.optim.lr_scheduler import ReduceLROnPlateau
# from sklearn.metrics import accuracy_score, precision_score, recall_score
# import  warnings
# import os
# from sklearn.metrics import classification_report, f1_score
#
# warnings.filterwarnings("ignore")  # 忽略警告信息
#
# # 数据加载和预处理
# neg = pd.read_excel('data/neg.xls', header=None)
# pos = pd.read_excel('data/pos.xls', header=None)
#
# pos['words'] = pos[0].apply(lambda x: jieba.lcut(str(x)))
# neg['words'] = neg[0].apply(lambda x: jieba.lcut(str(x)))
#
# texts = np.concatenate((pos['words'], neg['words']))
# labels = np.concatenate((np.ones(len(pos)), np.zeros(len(neg))))
#
# # BERT 模型准备
# tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
# model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=2)
#
# def encode_text(texts):
#     return tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=128)
#
# train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)
#
# train_encodings = encode_text([' '.join(text) for text in train_texts])
# val_encodings = encode_text([' '.join(text) for text in val_texts])
#
# class SentimentDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels
#
#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx], dtype=torch.long) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
#         return item
#
#     def __len__(self):
#         return len(self.labels)
#
# train_dataset = SentimentDataset(train_encodings, train_labels)
# val_dataset = SentimentDataset(val_encodings, val_labels)
#
# # 计算多个指标
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=1)
#
#     report = classification_report(labels, predictions, output_dict=True)
#     accuracy = report['accuracy']
#     precision = report['macro avg']['precision']
#     recall = report['macro avg']['recall']
#     f1 = report['macro avg']['f1-score']
#
#     return {
#         "accuracy": accuracy,
#         "precision": precision,
#         "recall": recall,
#         "f1": f1,
#         "loss": float('nan'),  # 这里可能需要根据具体需求调整
#     }
#
# # 设置优化器和学习率调整器
# optimizer = AdamW(model.parameters(), lr=2e-5)
# scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True, min_lr=1e-7)
#
# # 训练参数
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=3,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=64,
#     warmup_steps=500,
#     weight_decay=0.01,
#     evaluation_strategy="epoch",
#     logging_dir='./logs',
#     lr_scheduler_type="reduce_lr_on_plateau",
#     metric_for_best_model="loss",
# )
#
# # 初始化 Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     compute_metrics=compute_metrics,
# )
#
# # 开始训练
# trainer.train()
#
# # 保存模型
# model.save_pretrained('bert_sentiment_model')
# tokenizer.save_pretrained('bert_sentiment_model')
#
# # 输出训练指标表格
# log_history = trainer.state.log_history
#
# # 初始化一个列表来保存每一epoch的指标
# metrics = []
#
# for log_item in log_history:
#     # 确保该日志条目包含训练或评估的指标
#     if 'loss' in log_item or 'eval_loss' in log_item:
#         # 记录 epoch，如果有的话
#         epoch = log_item.get('epoch', None)
#         # 获取 Training Loss
#         train_loss = log_item.get('loss', None)
#         # 获取 Validation Loss
#         eval_loss = log_item.get('eval_loss', None)
#         # 获取其他指标
#         accuracy = log_item.get('eval_accuracy', None)
#         precision = log_item.get('eval_precision', None)
#         recall = log_item.get('eval_recall', None)
#         f1 = log_item.get('eval_f1', None)
#
#         # 把当前 Epoch 的指标收集到字典中
#         metrics.append({
#             'cust-epoch': epoch,
#             'train_loss': train_loss,
#             'eval_loss': eval_loss,
#             'Accuracy': accuracy,
#             'Precision': precision,
#             'Recall': recall,
#             'F1 Score': f1
#         })
#
# # 把列表转换为 Pandas DataFrame
# metrics_df = pd.DataFrame(metrics)
#
# # 使用 pd.set_option 来设置表格显示的列宽和宽度，避免省略号
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)
#
# print("Model training metrics table:")
# print(metrics_df)
#
# # 情感预测部分
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)
#
# def bert_predict(string):
#     inputs = tokenizer(string, return_tensors='pt', padding=True, truncation=True, max_length=128)
#     inputs = {k: v.to(device) for k, v in inputs.items()}
#     with torch.no_grad():
#         outputs = model(**inputs)
#         logits = outputs.logits
#         predicted_class = torch.argmax(logits).item()
#     sentiment = '积极' if predicted_class == 1 else '消极'
#     print(f"{string} [{sentiment}]")
#     return sentiment
#
# string = '还不错，符合需求'
# pred_result = bert_predict(string)
# print(f"预测结果: {pred_result}")

In [8]:
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# from sklearn.model_selection import train_test_split
# import pandas as pd
# import numpy as np
# import jieba
# import torch
# from torch.optim import AdamW
# from torch.optim.lr_scheduler import ReduceLROnPlateau
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# import warnings
# import os
#
# warnings.filterwarnings("ignore")  # 忽略警告信息
#
# # 数据加载和预处理
# neg = pd.read_excel('data/neg.xls', header=None)
# pos = pd.read_excel('data/pos.xls', header=None)
#
# pos['words'] = pos[0].apply(lambda x: jieba.lcut(str(x)))
# neg['words'] = neg[0].apply(lambda x: jieba.lcut(str(x)))
#
# texts = np.concatenate((pos['words'], neg['words']))
# labels = np.concatenate((np.ones(len(pos)), np.zeros(len(neg))))
#
# # BERT 模型准备
# tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
# model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=2)
#
# def encode_text(texts):
#     return tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=128)
#
# train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)
#
# train_encodings = encode_text([' '.join(text) for text in train_texts])
# val_encodings = encode_text([' '.join(text) for text in val_texts])
#
# class SentimentDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels
#
#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx], dtype=torch.long) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
#         return item
#
#     def __len__(self):
#         return len(self.labels)
#
# train_dataset = SentimentDataset(train_encodings, train_labels)
# val_dataset = SentimentDataset(val_encodings, val_labels)
#
# # 计算多个指标
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=1)
#
#     accuracy = accuracy_score(labels, predictions)
#     precision = precision_score(labels, predictions, average='binary')
#     recall = recall_score(labels, predictions, average='binary')
#     f1 = f1_score(labels, predictions, average='binary')
#
#     return {
#         "accuracy": accuracy,
#         "precision": precision,
#         "recall": recall,
#         "f1": f1,
#     }
#
# # 设置优化器和学习率调整器
# optimizer = AdamW(model.parameters(), lr=2e-5)
# scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True, min_lr=1e-7)
#
# # 训练参数
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=3,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=64,
#     warmup_steps=500,
#     weight_decay=0.01,
#     evaluation_strategy="epoch",
#     logging_dir='./logs',
#     lr_scheduler_type="reduce_lr_on_plateau",
#     metric_for_best_model="accuracy",  # 使用准确率作为最佳模型的评估指标
# )
#
# # 初始化 Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     compute_metrics=compute_metrics,
# )
#
# # 开始训练
# trainer.train()
#
# # 保存模型
# model.save_pretrained('bert_sentiment_model')
# tokenizer.save_pretrained('bert_sentiment_model')
#
# # 输出训练指标表格
# log_history = trainer.state.log_history
#
# # 初始化一个列表来保存每一epoch的指标
# metrics = []
#
# for log_item in log_history:
#     # 确保该日志条目包含训练或评估的指标
#     if 'loss' in log_item or 'eval_loss' in log_item:
#         # 记录 epoch，如果有的话
#         epoch = log_item.get('epoch', None)
#         # 获取 Training Loss
#         train_loss = log_item.get('loss', None)
#         # 获取 Validation Loss
#         eval_loss = log_item.get('eval_loss', None)
#         # 获取其他指标
#         accuracy = log_item.get('eval_accuracy', None)
#         precision = log_item.get('eval_precision', None)
#         recall = log_item.get('eval_recall', None)
#         f1 = log_item.get('eval_f1', None)
#
#         # 把当前 Epoch 的指标收集到字典中
#         metrics.append({
#             'cust-epoch': epoch,
#             'train_loss': train_loss,
#             'eval_loss': eval_loss,
#             'Accuracy': accuracy,
#             'Precision': precision,
#             'Recall': recall,
#             'F1 Score': f1
#         })
#
# # 把列表转换为 Pandas DataFrame
# metrics_df = pd.DataFrame(metrics)
#
# # 使用 pd.set_option 来设置表格显示的列宽和宽度，避免省略号
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)
#
# print("Model training metrics table:")
# print(metrics_df)
#
# # 情感预测部分
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)
#
# def bert_predict(string):
#     inputs = tokenizer(string, return_tensors='pt', padding=True, truncation=True, max_length=128)
#     inputs = {k: v.to(device) for k, v in inputs.items()}
#     with torch.no_grad():
#         outputs = model(**inputs)
#         logits = outputs.logits
#         predicted_class = torch.argmax(logits).item()
#     sentiment = '积极' if predicted_class == 1 else '消极'
#     print(f"{string} [{sentiment}]")
#     return sentiment
#
# string = '还不错，符合需求'
# pred_result = bert_predict(string)
# print(f"预测结果: {pred_result}")

In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import jieba
import torch
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
import os

warnings.filterwarnings("ignore")  # 忽略警告信息

# 数据加载和预处理
# 假设每个情感类别对应一个txt文件，文件名分别为：like.txt、dislike.txt、happy.txt、sad.txt、angry.txt、surprised.txt、afraid.txt、positive.txt、negative.txt、neutral.txt
# 每个文件中包含对应情感的文本数据，每行一个文本

# 定义情感类别和对应的文件名
emotions = ['喜欢', '厌恶', '开心', '难过', '愤怒', '惊讶', '害怕', '积极', '消极', '中性']
file_names = ['like.txt', 'dislike.txt', 'happy.txt', 'sad.txt', 'angry.txt', 'surprised.txt', 'afraid.txt', 'positive.txt', 'negative.txt', 'neutral.txt']

# 加载数据
texts = []
labels = []
for idx, file_name in enumerate(file_names):
    with open(os.path.join('data', file_name), 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                texts.append(line)
                labels.append(idx)

# 将文本进行分词处理
texts = [jieba.lcut(text) for text in texts]

# BERT 模型准备
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=len(emotions))

def encode_text(texts):
    return tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=128)

# 划分训练集和验证集
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)

# 对文本进行编码
train_encodings = encode_text([' '.join(text) for text in train_texts])
val_encodings = encode_text([' '.join(text) for text in val_texts])

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx], dtype=torch.long) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

# 计算多个指标
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')
    f1 = f1_score(labels, predictions, average='macro')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

# 设置优化器和学习率调整器
optimizer = AdamW(model.parameters(), lr=2e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True, min_lr=1e-7)

# 训练参数
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    lr_scheduler_type="reduce_lr_on_plateau",
    metric_for_best_model="accuracy",  # 使用准确率作为最佳模型的评估指标
)

# 初始化 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# 开始训练
trainer.train()

# 保存模型
model.save_pretrained('bert_sentiment_model')
tokenizer.save_pretrained('bert_sentiment_model')

# 输出训练指标表格
log_history = trainer.state.log_history

# 初始化一个列表来保存每一epoch的指标
metrics = []

for log_item in log_history:
    # 确保该日志条目包含训练或评估的指标
    if 'loss' in log_item or 'eval_loss' in log_item:
        # 记录 epoch，如果有的话
        epoch = log_item.get('epoch', None)
        # 获取 Training Loss
        train_loss = log_item.get('loss', None)
        # 获取 Validation Loss
        eval_loss = log_item.get('eval_loss', None)
        # 获取其他指标
        accuracy = log_item.get('eval_accuracy', None)
        precision = log_item.get('eval_precision', None)
        recall = log_item.get('eval_recall', None)
        f1 = log_item.get('eval_f1', None)

        # 把当前 Epoch 的指标收集到字典中
        metrics.append({
            'cust-epoch': epoch,
            'train_loss': train_loss,
            'eval_loss': eval_loss,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1
        })

# 把列表转换为 Pandas DataFrame
metrics_df = pd.DataFrame(metrics)

# 使用 pd.set_option 来设置表格显示的列宽和宽度，避免省略号
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

print("Model training metrics table:")
print(metrics_df)

# 情感预测部分
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def bert_predict(string):
    inputs = tokenizer(string, return_tensors='pt', padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits).item()
    sentiment = emotions[predicted_class]
    print(f"{string} [{sentiment}]")
    return sentiment

string = '在这个复杂多变的社会里，成年人之间的友谊似乎越来越像一场精心策划的交易。它不再是小时候那种纯粹的、毫无保留的情感连接，而是建立在利益、需求和相互利用基础上的一种微妙平衡。'
pred_result = bert_predict(string)
print(f"预测结果: {pred_result}")

/Users/grace/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/_w/hfqvgqcd4rj79tq5flh6k91h0000gn/T/jieba.cache
Loading model cost 0.315 seconds.
Prefix dict has been built successfully.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.593145,0.421000,0.437085,0.424197,0.398872
2,1.554100,1.558200,0.451000,0.440333,0.457524,0.431725
3,1.554100,1.610294,0.452000,0.457239,0.449866,0.444297


Model training metrics table:
   cust-epoch  train_loss  eval_loss  Accuracy  Precision    Recall  F1 Score
0         1.0         NaN   1.593145     0.421   0.437085  0.424197  0.398872
1         2.0      1.5541        NaN       NaN        NaN       NaN       NaN
2         2.0         NaN   1.558200     0.451   0.440333  0.457524  0.431725
3         3.0         NaN   1.610294     0.452   0.457239  0.449866  0.444297
在这个复杂多变的社会里，成年人之间的友谊似乎越来越像一场精心策划的交易。它不再是小时候那种纯粹的、毫无保留的情感连接，而是建立在利益、需求和相互利用基础上的一种微妙平衡。 [消极]
预测结果: 消极


In [5]:
import json
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import jieba
import torch
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
import os

warnings.filterwarnings("ignore")  # 忽略警告信息

# 数据加载和预处理
# 假设数据集保存在train.txt文件中，格式为JSON格式，每行一个样本
# 样本格式：{"id": 21, "content": "4…这都不是偶然的事情，这是他明白这个事实真相，对生死看得很淡薄，知道要认真努力修行，所以死了之后，自己可以作主，自己可以选择到哪一道去。如果在临终时，一慌一乱，你对身体很留恋，对家亲眷属很留恋，那你往往就到三途去了！", "label": "fear"}

# 定义情感类别和对应的标签映射
emotions = ['like', 'disgust', 'happiness', 'sadness', 'anger', 'fear']
label_to_id = {label: idx for idx, label in enumerate(emotions)}

# 加载数据
texts = []
labels = []
with open('data/train.txt', 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        text = data['content']
        label = data['label']
        if label in label_to_id:
            texts.append(text)
            labels.append(label_to_id[label])

# 将文本进行分词处理
texts = [jieba.lcut(text) for text in texts]

# BERT 模型准备
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=len(emotions))

def encode_text(texts):
    return tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=128)

# 划分训练集和验证集
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)

# 对文本进行编码
train_encodings = encode_text([' '.join(text) for text in train_texts])
val_encodings = encode_text([' '.join(text) for text in val_texts])

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx], dtype=torch.long) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

# 计算多个指标
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')
    f1 = f1_score(labels, predictions, average='macro')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

# 设置优化器和学习率调整器
optimizer = AdamW(model.parameters(), lr=2e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True, min_lr=1e-7)

# 训练参数
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    lr_scheduler_type="reduce_lr_on_plateau",
    metric_for_best_model="accuracy",  # 使用准确率作为最佳模型的评估指标
)

# 初始化 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# 开始训练
trainer.train()

# 保存模型
model.save_pretrained('bert_sentiment_model')
tokenizer.save_pretrained('bert_sentiment_model')

# 输出训练指标表格
log_history = trainer.state.log_history

# 初始化一个列表来保存每一epoch的指标
metrics = []

for log_item in log_history:
    # 确保该日志条目包含训练或评估的指标
    if 'loss' in log_item or 'eval_loss' in log_item:
        # 记录 epoch，如果有的话
        epoch = log_item.get('epoch', None)
        # 获取 Training Loss
        train_loss = log_item.get('loss', None)
        # 获取 Validation Loss
        eval_loss = log_item.get('eval_loss', None)
        # 获取其他指标
        accuracy = log_item.get('eval_accuracy', None)
        precision = log_item.get('eval_precision', None)
        recall = log_item.get('eval_recall', None)
        f1 = log_item.get('eval_f1', None)

        # 把当前 Epoch 的指标收集到字典中
        metrics.append({
            'cust-epoch': epoch,
            'train_loss': train_loss,
            'eval_loss': eval_loss,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1
        })

# 把列表转换为 Pandas DataFrame
metrics_df = pd.DataFrame(metrics)

# 使用 pd.set_option 来设置表格显示的列宽和宽度，避免省略号
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

print("Model training metrics table:")
print(metrics_df)

# 情感预测部分
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def bert_predict(string):
    inputs = tokenizer(string, return_tensors='pt', padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits).item()
    sentiment = emotions[predicted_class]
    print(f"{string} [{sentiment}]")
    return sentiment

string = '还不错，符合需求'
pred_result = bert_predict(string)
print(f"预测结果: {pred_result}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 